<a href="https://colab.research.google.com/github/arg-m/python/blob/main/%D0%91%D0%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Краснова Юлия, 09-313

In [ ]:
import sqlite3
import os
from datetime import datetime

# Подключение к базе данных (создание файла library.db)
if os.path.exists('library.db'):
    os.remove('library.db')

conn = sqlite3.connect('library.db')
cursor = conn.cursor()

# 1. Создание таблицы books
cursor.execute('''
CREATE TABLE IF NOT EXISTS books (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT NOT NULL,
    author TEXT NOT NULL,
    year_published INTEGER,
    genre TEXT
)
''')

# 2. Вставка данных
books_data = [
    ("The Great Gatsby", "F. Scott Fitzgerald", 1925, "Fiction"),
    ("1984", "George Orwell", 1949, "Dystopian"),
    ("To Kill a Mockingbird", "Harper Lee", 1960, "Classic")
]
cursor.executemany('''
INSERT INTO books (title, author, year_published, genre) VALUES (?, ?, ?, ?)
''', books_data)

# 3. Выборка всех данных
cursor.execute('SELECT * FROM books')
print("Все книги:")
for row in cursor.fetchall():
    print(row)

# 4. Фильтрация данных (книги после 1950 года)
cursor.execute('SELECT * FROM books WHERE year_published > 1950')
print("\nКниги, опубликованные после 1950 года:")
for row in cursor.fetchall():
    print(row)

# 5. Использование оператора LIKE (названия начинаются с "T")
cursor.execute('SELECT * FROM books WHERE title LIKE "T%"')
print("\nКниги, название которых начинается с 'T':")
for row in cursor.fetchall():
    print(row)

# 6. Сортировка данных (по году публикации)
cursor.execute('SELECT * FROM books ORDER BY year_published ASC')
print("\nКниги, отсортированные по году публикации:")
for row in cursor.fetchall():
    print(row)

# 7. Ограничение количества строк (первые две книги, отсортированные по названию)
cursor.execute('SELECT * FROM books ORDER BY title LIMIT 2')
print("\nПервые две книги, отсортированные по названию:")
for row in cursor.fetchall():
    print(row)

# 8. Агрегатные функции (общее количество книг)
cursor.execute('SELECT COUNT(*) FROM books')
total_books = cursor.fetchone()[0]
print(f"\nОбщее количество книг: {total_books}")

# 9. Группировка данных (количество книг для каждого жанра)
cursor.execute('SELECT genre, COUNT(*) FROM books GROUP BY genre')
print("\nКоличество книг для каждого жанра:")
for row in cursor.fetchall():
    print(row)

# 10. Обновление данных (обновить год публикации книги "1984" на 1948)
cursor.execute('UPDATE books SET year_published = 1948 WHERE title = "1984"')

# 11. Удаление данных (удалить книгу "The Great Gatsby")
cursor.execute('DELETE FROM books WHERE title = "The Great Gatsby"')

# 12. Создание индекса (для столбца author)
cursor.execute('CREATE INDEX IF NOT EXISTS idx_author ON books(author)')

# 13. Проверка существования индекса
cursor.execute("SELECT name FROM sqlite_master WHERE type='index' AND name='idx_author'")
index_exists = cursor.fetchone()
print(f"\nИндекс для столбца author {'существует' if index_exists else 'не существует'}.")

# 14. Триггер для логирования
cursor.execute('''
CREATE TABLE IF NOT EXISTS logs (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    message TEXT
)
''')
cursor.execute('''
CREATE TRIGGER IF NOT EXISTS log_insert AFTER INSERT ON books
BEGIN
    INSERT INTO logs (message) VALUES ('Добавлена новая книга: ' || NEW.title);
END;
''')

# 15. Транзакции
try:
    conn.execute('BEGIN TRANSACTION')
    # Добавление новой книги
    cursor.execute('INSERT INTO books (title, author, year_published, genre) VALUES (?, ?, ?, ?)',
                   ("New Book", "Author Name", 2023, "Fantasy"))
    # Обновление года публикации другой книги
    cursor.execute('UPDATE books SET year_published = 1961 WHERE title = "To Kill a Mockingbird"')
    # Отмена изменений (раскомментируйте следующую строку для отмены транзакции)
    # raise Exception("Ошибка транзакции!")
    conn.commit()
except Exception as e:
    conn.rollback()
    print(f"Транзакция отменена: {e}")

# 16. Пользовательская функция (возраст книги)
def book_age(year_published):
    current_year = datetime.now().year
    return current_year - year_published

conn.create_function("book_age", 1, book_age)
cursor.execute('SELECT title, book_age(year_published) AS age FROM books')
print("\nВозраст книг:")
for row in cursor.fetchall():
    print(row)

# 17. Представления (modern_books)
cursor.execute('''
CREATE VIEW IF NOT EXISTS modern_books AS
SELECT * FROM books WHERE year_published > 1950
''')

# 18. Удаление таблицы и проверка её существования
cursor.execute('DROP TABLE IF EXISTS books')
cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='books'")
table_exists = cursor.fetchone()
print(f"\nТаблица books {'существует' if table_exists else 'не существует'}.")

# Пересоздаем таблицу books, если она была удалена
cursor.execute('''
CREATE TABLE IF NOT EXISTS books (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT NOT NULL,
    author TEXT NOT NULL,
    year_published INTEGER,
    genre TEXT
)
''')

# Вставляем данные заново (если таблица была удалена)
books_data = [
    ("The Great Gatsby", "F. Scott Fitzgerald", 1925, "Fiction"),
    ("1984", "George Orwell", 1949, "Dystopian"),
    ("To Kill a Mockingbird", "Harper Lee", 1960, "Classic")
]
cursor.executemany('''
INSERT INTO books (title, author, year_published, genre) VALUES (?, ?, ?, ?)
''', books_data)

# 19. Композитный индекс
cursor.execute('CREATE INDEX IF NOT EXISTS idx_author_year ON books(author, year_published)')
print("\nКомпозитный индекс создан.")

# 20. Удаление базы данных
conn.close()
import os
if os.path.exists('library.db'):
    os.remove('library.db')
    print("\nБаза данных library.db удалена.")
else:
    print("\nБаза данных library.db не найдена.")


задание 21 для 16 варианта

In [ ]:
import sqlite3
from datetime import datetime

# Подключение к базе данных (создание файла library.db)
conn = sqlite3.connect('library.db')
cursor = conn.cursor()

# 1. Создание таблицы DigitalResource
cursor.execute('''
CREATE TABLE IF NOT EXISTS DigitalResource (
    resource_id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT NOT NULL,
    author TEXT NOT NULL,
    file_size REAL NOT NULL,
    is_available BOOLEAN NOT NULL DEFAULT 1
)
''')

# 2. Создание таблицы Ebook (наследуется от DigitalResource)
cursor.execute('''
CREATE TABLE IF NOT EXISTS Ebook (
    resource_id INTEGER PRIMARY KEY,
    page_count INTEGER NOT NULL,
    FOREIGN KEY (resource_id) REFERENCES DigitalResource(resource_id)
)
''')

# 3. Создание таблицы Audiobook (наследуется от DigitalResource)
cursor.execute('''
CREATE TABLE IF NOT EXISTS Audiobook (
    resource_id INTEGER PRIMARY KEY,
    duration REAL NOT NULL,
    FOREIGN KEY (resource_id) REFERENCES DigitalResource(resource_id)
)
''')

# 4. Создание таблицы Movie (наследуется от DigitalResource)
cursor.execute('''
CREATE TABLE IF NOT EXISTS Movie (
    resource_id INTEGER PRIMARY KEY,
    resolution TEXT NOT NULL,
    FOREIGN KEY (resource_id) REFERENCES DigitalResource(resource_id)
)
''')

# 5. Создание таблицы MusicTrack (наследуется от DigitalResource)
cursor.execute('''
CREATE TABLE IF NOT EXISTS MusicTrack (
    resource_id INTEGER PRIMARY KEY,
    genre TEXT NOT NULL,
    FOREIGN KEY (resource_id) REFERENCES DigitalResource(resource_id)
)
''')

# 6. Создание таблицы User
cursor.execute('''
CREATE TABLE IF NOT EXISTS User (
    user_id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    email TEXT NOT NULL UNIQUE
)
''')

# 7. Создание таблицы Subscription
cursor.execute('''
CREATE TABLE IF NOT EXISTS Subscription (
    subscription_id INTEGER PRIMARY KEY AUTOINCREMENT,
    user_id INTEGER NOT NULL,
    sub_type TEXT NOT NULL,
    expiry_date TEXT NOT NULL,
    FOREIGN KEY (user_id) REFERENCES User(user_id)
)
''')

# 8. Создание таблицы DownloadHistory (история загрузок)
cursor.execute('''
CREATE TABLE IF NOT EXISTS DownloadHistory (
    download_id INTEGER PRIMARY KEY AUTOINCREMENT,
    resource_id INTEGER NOT NULL,
    user_id INTEGER NOT NULL,
    download_date TEXT NOT NULL,
    FOREIGN KEY (resource_id) REFERENCES DigitalResource(resource_id),
    FOREIGN KEY (user_id) REFERENCES User(user_id)
)
''')

# Функция для добавления цифрового ресурса
def add_digital_resource(title, author, file_size, resource_type, **kwargs):
    cursor.execute('''
    INSERT INTO DigitalResource (title, author, file_size, is_available)
    VALUES (?, ?, ?, 1)
    ''', (title, author, file_size))
    resource_id = cursor.lastrowid

    if resource_type == 'ebook':
        cursor.execute('''
        INSERT INTO Ebook (resource_id, page_count)
        VALUES (?, ?)
        ''', (resource_id, kwargs.get('page_count')))
    elif resource_type == 'audiobook':
        cursor.execute('''
        INSERT INTO Audiobook (resource_id, duration)
        VALUES (?, ?)
        ''', (resource_id, kwargs.get('duration')))
    elif resource_type == 'movie':
        cursor.execute('''
        INSERT INTO Movie (resource_id, resolution)
        VALUES (?, ?)
        ''', (resource_id, kwargs.get('resolution')))
    elif resource_type == 'music':
        cursor.execute('''
        INSERT INTO MusicTrack (resource_id, genre)
        VALUES (?, ?)
        ''', (resource_id, kwargs.get('genre')))

    conn.commit()
    return resource_id

# Функция для добавления пользователя
def add_user(name, email):
    cursor.execute('''
    INSERT INTO User (name, email)
    VALUES (?, ?)
    ''', (name, email))
    conn.commit()
    return cursor.lastrowid

# Функция для добавления подписки
def add_subscription(user_id, sub_type, expiry_date):
    cursor.execute('''
    INSERT INTO Subscription (user_id, sub_type, expiry_date)
    VALUES (?, ?, ?)
    ''', (user_id, sub_type, expiry_date))
    conn.commit()

# Функция для записи истории загрузок
def log_download(resource_id, user_id):
    download_date = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    cursor.execute('''
    INSERT INTO DownloadHistory (resource_id, user_id, download_date)
    VALUES (?, ?, ?)
    ''', (resource_id, user_id, download_date))
    conn.commit()

# Функция для получения всех ресурсов
def get_all_resources():
    cursor.execute('SELECT * FROM DigitalResource')
    return cursor.fetchall()

# Функция для получения всех пользователей
def get_all_users():
    cursor.execute('SELECT * FROM User')
    return cursor.fetchall()

# Пример использования
if __name__ == "__main__":
    # Добавление ресурсов
    ebook_id = add_digital_resource("Война и Мир", "Лев Толстой", 2.5, "ebook", page_count=1225)
    audiobook_id = add_digital_resource("Мастер и Маргарита", "Михаил Булгаков", 300.0, "audiobook", duration=600.0)

    # Добавление пользователей
    user1_id = add_user("Иван Иванов", "ivan@example.com")
    user2_id = add_user("Мария Петрова", "maria@example.com")

    # Добавление подписок
    add_subscription(user1_id, "basic", "2024-12-31")
    add_subscription(user2_id, "premium", "2023-10-15")

    # Логирование загрузок
    log_download(ebook_id, user1_id)
    log_download(audiobook_id, user2_id)

    # Вывод данных
    print("Все ресурсы:")
    for resource in get_all_resources():
        print(resource)

    print("\nВсе пользователи:")
    for user in get_all_users():
        print(user)

# Закрытие соединения с базой данных
conn.close()